# Inference on test data (2019-2020)

### Import libraries

In [1]:
import os
import toml
import numpy as np
import xarray as xr
from tqdm import tqdm
from typing import Any

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.markers as markers
from mpl_toolkits.axes_grid1 import make_axes_locatable    

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter)


Add to system path the parent folder in order to be able to import the **Fires** library:

In [2]:
import sys
_pth = "../"
if _pth not in sys.path:
	sys.path.append(_pth)

Import from **Fires** library what is necessary to build the Inference workflow:

In [3]:

import Fires
from Fires._datasets.torch_dataset import FireDataset

from Fires._macros.macros import (
    CONFIG,
    DATA_DIR,
    LOG_DIR,
    NEW_DS_PATH,
    RUN_DIR,
    SCALER_DIR,
    TORCH_CFG
)

from Fires._models.unetpp import UnetPlusPlus as UPP

from Fires._scalers.minmax import MinMaxScaler
from Fires._scalers.standard import StandardScaler

from Fires._utilities.configuration import load_global_config


/Users/emanueledonno/VSCode/CMCC/ML4Fires/config
/Users/emanueledonno/VSCode/CMCC/ML4Fires/digital_twin_notebooks


/Users/emanueledonno/opt/anaconda3/envs/cmcc-torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiments and plots

### Plot functions

Define:
- the projection type
- the map extension
- the arrays with the latitudes and the longitudes that must used for axes

In [4]:
# define projection
datacrs = ccrs.PlateCarree()

# define map extent
extent_args=dict(extents = [-180, 180, -90, 90], crs=datacrs)

# define latitudes and longitudes array that must be used for axes
latitudes = np.arange(-60, 90, 30)
longitudes = np.arange(-160, 180, 80)

Function used to draw map features:

In [5]:
def draw_features(ax:Any):
	"""
	This function adds several geographical features to the map using Cartopy features:

    * Political borders: outlines country borders with a solid black line style (':') and a linewidth of 0.5.
    * Oceans: outlines the ocean regions with a solid black line style ('-') and a linewidth of 0.8.
    * Lakes: outlines lakes with a solid black line style ('-') and a linewidth of 0.8.
    * Rivers: outlines rivers with a solid black line style ('-') and a linewidth of 0.8.
    * Coastlines: adds high-resolution coastlines (50 meters) to the map with a higher zorder (3) for better visibility.

    **Note:** 
        * Land is not explicitly added in this function. 
        * Adding a background image using `ax.stock_img()` is not implemented. 

	Parameters
	----------
	ax : Any
		The matplotlib axis object to add the features to.

	Returns
	-------
	ax : Any
		The modified matplotlib axis object.

	
    
	"""
	# political borders
	ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.5, edgecolor='k')
	# add ocean
	ax.add_feature(cfeature.OCEAN, linestyle='-',linewidth=0.8, edgecolor='k')
	# add lakes
	ax.add_feature(cfeature.LAKES, linestyle='-',linewidth=0.8, edgecolor='k')
	# add rivers
	ax.add_feature(cfeature.RIVERS, linestyle='-',linewidth=0.8, edgecolor='k')
	# add land
	# ax.add_feature(cfeature.LAND, zorder=1, edgecolor='k')
	# add coastlines
	ax.coastlines(resolution='50m', zorder=3)
	
	# add stock image
	# ax.stock_img()

	return ax

Function used to highlight specific burned areas points on map:

In [6]:
def highlight_ba(ax:Any, y:float, x:float, color:str):
	"""
	Plots lines and a circle to highlight a specific point on a map.

	Parameters
	----------
	ax : Any
		The matplotlib axis object where to plot the elements.
	y : float
		The latitude value of the point to highlight.
	x : float
		The longitude value of the point to highlight.
	color : str
		The color to use for the lines and circle.

	Returns
	-------
	ax : Any 
		The modified matplotlib axis object.
	"""
	
	# plot lines corresponding to latitude and longitude value of burned areas
	ax.axhline(y=y, color=color, linewidth=3, zorder=3, linestyle=':')
	ax.text(-181.5, np.round(y, 2), f'{np.round(y, 2)}', color=color, fontweight='bold', size=50, ha='center', va='center', rotation=90)
	
	ax.axvline(x=x, color=color, linewidth=3, zorder=3, linestyle=':')
	ax.text(np.round(x, 2), -90.5, f'{np.round(x, 2)}', color=color, fontweight='bold', size=50, ha='center', va='top')
	
	# plot cicle around the pixel with the value of burned areas
	circle = plt.Circle((x, y), 1, color=color, linewidth=5, fill=False, zorder=3)
	ax.add_patch(circle)
	
	return ax

Funtion to set the axis labels, ticks, and formatters for latitude and longitude:

In [7]:
def set_axis(ax, is_y:bool, latlon_vals, gl):
	"""
	Sets the axis labels, ticks, and formatters for latitude or longitude on a map.

	Parameters
	----------
	ax : Any
		The matplotlib axis object to modify.
	is_y : bool
		True if setting the y-axis, False for x-axis.
	latlon_vals : np.array
		The list of latitude or longitude values for the axis.
	gl : Any
		The gridlines object for the map.

	Returns
	-------
	ax : Any
		The modified axis object.
	"""
	values = latlon_vals
	if is_y:
		lat_formatter = LatitudeFormatter()
		ax.yaxis.set_major_formatter(lat_formatter)
		ax.yaxis.set_major_locator(mticker.FixedLocator(values))
		ax.set_yticklabels(values, fontweight='bold', size=50, rotation=90)
		ax.set_yticks(values)
		gl.xlocator = mticker.FixedLocator(values)
		gl.xlines = False

	else:
		lon_formatter = LongitudeFormatter(zero_direction_label=True)
		ax.xaxis.set_major_formatter(lon_formatter)
		ax.xaxis.set_major_locator(mticker.FixedLocator(values))
		ax.set_xticklabels(values, fontweight='bold', size=50)
		ax.set_xticks(values)
		gl.ylocator = mticker.FixedLocator(values)
		gl.ylines = False

	return ax

Function to draw Tropic of Cancer, Equator, and Tropic of Capricorn on a map:

In [8]:
def draw_tropics_and_equator(ax):
	"""
	Plots lines representing the Tropic of Cancer, Equator, and Tropic of Capricorn on a map.

	Parameters
	----------
	ax : Any
		The matplotlib axis object where to plot the lines.

	Returns
	-------
	ax : Any: 
		The modified matplotlib axis object.
	"""
	ax.axhline(23.5, linestyle=':', color='blue', linewidth=0.7, label='Tropic of Cancer')
	ax.axhline(0.00, linestyle=':', color='black', linewidth=0.7, label='Equator')
	ax.axhline(-23.5, linestyle=':', color='blue', linewidth=0.7, label='Tropic of Capricorn')
	return ax

Function that generates a comprehensive map visualization of the burned areas data, highlighting minimum and maximum values alongside their confidence intervals:

In [9]:
def plot_dataset_map(
	avg_target_data:np.array,
	avg_data_on_lats:np.array,
	lowerbound_data:np.array,
	upperbound_data:np.array,
	lats:list,
	lons:list,
	title:str,
	cmap:str) -> None:
	"""
	Generates a comprehensive map visualization of a dataset, highlighting 
	minimum and maximum values alongside their confidence intervals.

	Parameters
	----------
	avg_target_data : np.array
		2D array containing the core data to be visualized as color intensity on the map.
		Missing values (NaN) are handled by setting the color to transparent.

	avg_data_on_lats : np.array
		1D array containing the average of the target data for each latitude value.
		This data is plotted as a line in a secondary subplot.

	lowerbound_data : np.array
		2D array containing the lower bound of the data (e.g., standard deviation or confidence interval) for each latitude and longitude.
		This data is used to shade the area around the average line in the secondary subplot.

	upperbound_data : np.array
		2D array containing the upper bound of the data for each latitude and longitude.
		Similar to `lowerbound_data`, it's used for shading the confidence interval in the secondary subplot.

	lats : list
		List containing the latitude values corresponding to the data.

	lons : list
		List containing the longitude values corresponding to the data.

	title : str
		The title to be displayed at the top of the plot.

	cmap : str
		The name of the colormap to use for visualizing the data on the map.
	
	Returns
	-------
	None
		Saves the figure as a high-resolution PNG image (300 dpi) but does not return anything.
	"""
	# define color
	color = 'darkred' #fc6742 #4296fc #990e0e
	
	# compute maximum along latitudes and longitudes and find index
	maximum_val = np.nanmax(avg_target_data)
	lat_idx_max, lon_idx_max  = np.where(avg_target_data==maximum_val)
	max_val_latitude = lats[lat_idx_max][0]
	max_val_longitude = lons[lon_idx_max][0]
	
	# compute minimum along latitudes and longitudes and find index
	minimum_val = np.nanmin(avg_target_data)
	lat_idx_min, lon_idx_min  = np.where(avg_target_data==minimum_val)
	min_val_latitude = lats[lat_idx_min][0]
	min_val_longitude = lons[lon_idx_min][0]
	
	# define fiure and subplots
	_, ax1 = plt.subplots(figsize=(90, 80), subplot_kw=dict(projection=datacrs), sharey=True)
	
	# set title of the plot
	ax1.set_title(title, fontweight='bold', size=80)
	
	# set x and y labels
	ax1.set_xlabel('Longitude [deg]', fontweight='bold', size=50)
	ax1.set_ylabel('Latitude [deg]', fontweight='bold', size=50)
	
	# set map extent
	ax1.set_extent(**extent_args)
	
	# plot map features such as borders, sea, lakes, rivers and background image
	ax1 = draw_features(ax=ax1)
	
	# plot data on the map
	cmap = plt.get_cmap(cmap)
	cmap.set_under((0, 0, 0, 0))
	h = ax1.pcolormesh(lons, lats, avg_target_data, transform=datacrs, cmap=cmap, zorder=3, alpha=0.5)
	
	# highlight pixel where the maximum value ov burned areas has been found and put a circle around it
	ax1 = highlight_ba(ax=ax1, y=max_val_latitude, x=max_val_longitude, color=color)
	
	# highlight pixel where the minimum value ov burned areas has been found and put a circle around it
	ax1 = highlight_ba(ax=ax1, y=min_val_latitude, x=min_val_longitude, color='green')
	
	# add grid lines for latitude and longitude
	gl = ax1.gridlines(crs=datacrs, draw_labels=False, linewidth=1.5, color='gray', alpha=0.5, linestyle='-', zorder=3)
	
	# define longitudes and set x ticks
	ax1 = set_axis(ax=ax1, is_y=False, latlon_vals=longitudes, gl=gl)
	
	# define latitudes and set y ticks
	ax1 = set_axis(ax=ax1, is_y=True, latlon_vals=latitudes, gl=gl)
	
	# define latitudes for tropics and equator
	ax1 = draw_tropics_and_equator(ax=ax1)
	
	# add subplot
	divider = make_axes_locatable(ax1)
	ax2 = divider.append_axes("right", size="10%", pad=0.5, axes_class=plt.Axes)
	
	# plot data
	ax2.plot(avg_data_on_lats, lats, color='red', linewidth=1)
	ax2.plot(upperbound_data, lats, alpha=0.3, color='black', linewidth=0.5)
	ax2.plot(lowerbound_data, lats, alpha=0.3, color='black', linewidth=0.5)
	
	# fill space between lines
	ax2.fill_betweenx(y=lats, x1=avg_data_on_lats, x2=upperbound_data, color='gray', alpha=0.15)
	ax2.fill_betweenx(y=lats, x1=avg_data_on_lats, x2=lowerbound_data, color='gray', alpha=0.15)
	
	# define latitudes for tropics (in degrees) and equator
	ax2 = draw_tropics_and_equator(ax=ax2)
	
	# plot max position
	ax2.axhline(max_val_latitude, color=color, linewidth=3)
	
	# plot min position
	ax2.axhline(min_val_latitude, color='green', linewidth=3)
	
	# set x label	
	ax2.set_xlabel(' Mean ', fontweight='bold', size=50, labelpad=50)
	
	# create list of max values
	ax2_vals = np.around([np.nanmin(lowerbound_data, axis=0), np.nanmax(avg_data_on_lats, axis=0), np.nanmax(upperbound_data, axis=0)], 2)
	
	# plot axes tick lines§
	for tick in ax2_vals:
		ax2.axvline(x=tick, color='blue', alpha=1, linewidth=1, linestyle=':')
		ax2.text(round(tick), -.005, f'{round(tick)}', color='blue', fontweight='bold', size=50, transform=ax2.get_xaxis_transform(), ha='center', va='top')

	ax2.text(0, -.005, '0', color='black', fontweight='bold', size=50, transform=ax2.get_xaxis_transform(), ha='center', va='top') 
	
	ax2.set_xticks([])
	ax2.set_yticks([])
	ax2.set_ylim(bottom=-90, top=90)
	ax2.margins(y=0)
	# ax2.autoscale_view(scaley=True)
		
	# add colorbar plot
	ax_cb = divider.append_axes("right", size="2%", pad=0.3, axes_class=plt.Axes)
	cbar = plt.colorbar(h, ax_cb)
	cbar.ax.tick_params(labelsize=30)
	cbar.ax.set_ylabel('Hectares', color='black', fontweight='bold', size=50, labelpad=50, rotation=270)
	
	plt.tight_layout()
	plt.savefig(f"./img/fcci {title}.png", dpi=300)
	# plt.clf()

### Experiments

The first step in the inference pipeline is to select the neural network configuration (**experiment**) and the trained model with that configuration, to generate the burned area maps with the test data.

In [14]:
# define list of folders with the experiments
experiment_paths = ['20240311_222733']

# define main path to experiments folder
main_path = os.path.join(os.path.dirname(os.getcwd()), "experiments")

# define single experiment path
single_path = os.path.join(main_path, str(experiment_paths[0]))
print(f"Experiment path: {single_path}")

# define a common dictionary with all the experiments
exp_dicts = dict()
for folder in experiment_paths:
	path = os.path.join(main_path, folder)
	for file in os.listdir(path):
		if file.endswith(".toml"):
			exp_args = toml.load(os.path.join(path, file))
			exp_dicts[file.split('.')[0]] = exp_args

Experiment path: /Users/emanueledonno/VSCode/CMCC/ML4Fires/experiments/20240311_222733


In [16]:
exp_dicts.keys()

dict_keys(['exp_5'])

This notebook showcases an example of using a preliminary version of the Digital Twin. In this first part, we will focus on creating the reference dataset for inference, which is a prerequisite to running the subsequent cells of the notebook. Steps:

1. **Download the SeasFireCube v3 dataset**:
	Download the [SeasFireCube v3](https://zenodo.org/records/8055879) dataset and save it in the `data` folder.
	
2. **Load the dataset with `xarray`**
	Import the xarray library and load the SeasFireCube v3 dataset saved in the `data` folder.

3. **Select variables**
	Use the experiment configuration `/experiments/20240311_222733/exp_5.toml` to identify the `drivers` and `targets` variables to be used in the model. Define these variables and select related data from previously loaded dataset.

4. **Expand the `lsm` variable**
	The `lsm` variable represents the land-sea mask. Expand this variable to cover the entire time range of the dataset.

5. **Select the years of interest**
	Select the years of interest for the analysis, in this case 2019 and 2020. Filter the dataset based on these years.

6. **Save the reference dataset**
	Save the filtered and prepared reference dataset to the data folder.

7. **Update the path in the configuration file**
	Open the experiment configuration file `/experiments/20240311_222733/exp_5.toml`. Update the value of the `path_to_zarr` key with the full path to the saved reference dataset in the data folder.

> [!NOTE]
> The SeasFireCube v3 dataset is large in size and may not be possible to upload directly to GitHub.
> Creating the reference dataset is a preliminary step required to run the subsequent cells of the notebook.
> Make sure you have downloaded and saved the SeasFireCube v3 dataset correctly to the data folder before proceeding.
> Verify that the path to the reference dataset is updated correctly in the experiment configuration file.


After successfully completing the steps described in this section, you will have created the reference dataset necessary to run the subsequent cells of the notebook and test the Digital Twin.

The Python code in the cell below iterates through experiments in a dictionary named `exp_dicts` and performs the following tasks for each experiment:
* Loads the experiment configuration.
* Loads a pre-trained model and its optimizer and scheduler.
* Defines a scaler to normalize the test data.
* Creates a torch dataset and data loader for the test data.
* Predicts burned areas using the loaded model.
* Loads real burned area data.
* Calculates statistics (mean, standard deviation) for both predicted and real burned areas.
* Calculates the difference between predicted and real burned areas.
* Plots the result maps for predicted, real, and difference data.

In [ ]:
for key in exp_dicts.keys():
	
	# define current experiment
	# key = list(exp_dict.keys())[0]
	curr_exp = exp_dicts[key]
	curr_exp.keys()
	
	#
	#										Define model
	#

	# define dictionary containing all model related features
	model_dict = curr_exp['model']
	# get model path
	last_model_path = model_dict['last_model'].split("wildfires/")[1]
	# load model from path
	last_model_dict = torch.load(last_model_path, map_location=torch.device('cpu'))
	# get model state
	lm_state = last_model_dict['model']
	# get model optimizer
	lm_optimizer = last_model_dict['optimizer']
	# get model scheduler
	lm_scheduler = last_model_dict['scheduler']

	# get model class
	mdl_cls = eval(model_dict['cls'])
	# get model arguments
	mdl_arg = model_dict['args']
	# create a dictionary with all model arguments
	args_ = dict()
	# define model arguments
	for k in mdl_arg.keys():
		args_[k] = eval(mdl_arg[k]) if k == "activation" else mdl_arg[k]
	# define mdel
	model = mdl_cls(**args_)
	
	# load weights
	model.load_state_dict(lm_state)
	# evaluate model
	model.eval()

	#
	#									Dataset for testing model
	#

	# define path to complete dataset
	dataset_path = "./data/seasfire_v03.zarr"
	
	# get land sea mask and substitute zeros with NaN values
	lsm = xr.open_zarr(dataset_path)['lsm'].values
	lsm[lsm == 0] = np.nan
	
	# get valid mask data and retrieve valid dates for test data
	valid_mask_data = xr.open_zarr(dataset_path)['fcci_ba_valid_mask'].sel(time=slice('2019', '2021'))
	valid_dates = [time for time in valid_mask_data.time.data if valid_mask_data.sel(time=str(time)) == 1]
	
	# define main path, main dataset and select valid dates
	main_path = "./data/sfv03_fcci.zarr"
	main_data = xr.open_zarr(main_path)
	main_data = main_data.sel(time = slice(str(valid_dates[0]), str(valid_dates[-1])))
	
	# get drivers and targets
	drivers = curr_exp['features']['drivers']
	targets = curr_exp['features']['targets']
	
	# define test datasets for drivers and targets
	driver_ds = main_data[drivers].load()
	target_ds = main_data[targets].load()
	
	# get training years list and define training data
	trn_years = curr_exp['dataset']['trn_years']
	trn_data = main_data[drivers].sel(time=slice(str(trn_years[0]), str(trn_years[-1])))
	
	#
	# 				Load mean and standard deviation maps and create scaler
	#

	# load mean and standard deviation point dataset in order to scale test data
	mean_ds = xr.load_dataset(curr_exp['scalers']['paths']['fcci_mean_point_map']).load()
	stdv_ds = xr.load_dataset(curr_exp['scalers']['paths']['fcci_stdv_point_map']).load()
	
	# define scaler for drivers
	scaler_cls = eval(curr_exp['scalers']['cls'].split("'")[1])
	x_scaler = scaler_cls(mean_ds=mean_ds, stdv_ds=stdv_ds, features=drivers)
	
	#
	#						Create torch dataset and data loader
	#
	
	# define FireDataset arguments and FireDataset torch dataset for test data
	fire_ds_args = dict(src=main_path, drivers=drivers, targets=targets)
	test_torch_ds = FireDataset(**fire_ds_args, years=list(range(2019, 2022)), scalers=[x_scaler, None])
	
	# define test data loader
	test_dl_args = dict(batch_size=1, shuffle=True, drop_last=curr_exp['trainer']['drop_reminder'])
	test_loader = torch.utils.data.DataLoader(test_torch_ds, **test_dl_args)

	
	#
	#								Predict burned areas
	#
	
	# define predictions list, predict data and store them
	preds = []
	with torch.no_grad():
		i = 0
		for data, _ in tqdm(test_loader):
			if i >= 92: break
			preds.append(model(data))
			i += 1
	
	# define predictions array as vertical stack of predictions list
	preds_arr = np.vstack(preds)
	
	# ------------------------------------------------------------------------------------
	
	# define max hectares value
	max_hectares = pow((111/4), 2) * 100
	
	# define latitudes and longitudes lists
	lats = target_ds.latitude.data	
	lons = target_ds.longitude.data
	print(lats[:5])
	print(lons[:5])
	
	# ------------------------------------------------------------------------------------
	
	# real target data (temporal mean)
	avg_real_on_time = target_ds.fcci_ba.mean(dim='time', skipna=True).data
	
	# mask mean dataset with the land sea mask and rescale to original size
	avg_real_descaled = avg_real_on_time * lsm * max_hectares
	
	# compute the average along latitudes and longitudes
	avg_real_on_lats = np.nanmean(avg_real_descaled, axis=1)
	avg_real_on_lons = np.nanmean(avg_real_descaled, axis=0)
	
	# real target data (temporal standard deviation)
	std_real_on_time = target_ds.fcci_ba.std(dim='time', skipna=True).data
	
	# mask standard deviation data with the land sea mask and rescale to original size
	std_real_descaled = std_real_on_time * lsm * max_hectares
	
	# compute the average along latitudes and longitudes
	std_real_on_lats = np.nanmean(std_real_descaled, axis=1)
	std_real_on_lons = np.nanmean(std_real_descaled, axis=0)
	
	# define upperbound and lowerbound data for plotting the average on latitudes
	real_upperbound = avg_real_on_lats + std_real_on_lats
	real_lowerbound = avg_real_on_lats - std_real_on_lats
	
	print(f" Mean real {avg_real_descaled.shape}")
	print(f" Max: {np.nanmax(avg_real_descaled)}")
	print(f" Min: {np.nanmin(avg_real_descaled)}")
	print(f" Lats: {np.nanmax(avg_real_on_lats)}")
	print(f" Lons: {np.nanmax(avg_real_on_lons)}\n")
	
	print(f" Stdv real {std_real_descaled.shape}")
	print(f" Max: {np.nanmax(std_real_descaled)}")
	print(f" Min: {np.nanmin(std_real_descaled)}")
	print(f" Lats: {np.nanmax(std_real_on_lats)}")
	print(f" Lons: {np.nanmax(std_real_on_lons)}\n")
	
	print(f"Upperbound: {np.nanmin(real_upperbound)} \t {np.nanmax(real_upperbound)}\n")
	print(f"Lowerbound: {np.nanmin(real_lowerbound)} \t {np.nanmax(real_lowerbound)}\n")
	
	__x_idx, __y_idx = np.where(avg_real_on_time==np.nanmax(avg_real_on_time))
	print(lats[__x_idx], lons[__y_idx])
	
	
	# ------------------------------------------------------------------------------------
	
	# compute temporal mean for prediciton
	avg_preds_on_time = np.nanmean(preds_arr, axis=0)[0, ...]
	
	# mask mean dataset with the land sea mask and rescale to original size
	avg_preds_descaled =  avg_preds_on_time * lsm * max_hectares
	
	# compute the average along latitudes and longitudes
	avg_preds_on_lats = np.nanmean(avg_preds_descaled, axis=1)
	avg_preds_on_lons = np.nanmean(avg_preds_descaled, axis=0)
	
	# compute temporal standard deviation for prediction
	std_preds_on_time = np.nanstd(preds_arr, axis=0)[0, ...]
	
	# mask standard deviation data with the land sea mask and rescale to original size
	std_preds_descaled = std_preds_on_time * lsm * max_hectares
	
	# compute the average along latitudes and longitudes
	std_preds_on_lats = np.nanmean(std_preds_descaled, axis=1)
	std_preds_on_lons = np.nanmean(std_preds_descaled, axis=0)
	
	# define upperbound and lowerbound data for plotting the average on latitudes
	preds_upperbound = avg_preds_on_lats + std_preds_on_lats
	preds_lowerbound = avg_preds_on_lats - std_preds_on_lats
	
	print(f"{preds_arr.shape} \n {len(preds)} \n {len(valid_dates)} \n ")
	
	print(f" Mean predictions {avg_preds_descaled.shape}")
	print(f" Max: {np.nanmax(avg_preds_descaled)}")
	print(f" Min: {np.nanmin(avg_preds_descaled)}")
	print(f" Lats: {np.nanmax(avg_preds_on_lats)}")
	print(f" Lons: {np.nanmax(avg_preds_on_lons)}\n")
	
	print(f" Stdv predictions {std_preds_descaled.shape}")
	print(f" Max: {np.nanmax(std_preds_descaled)}")
	print(f" Min: {np.nanmin(std_preds_descaled)}")
	print(f" Lats: {np.nanmax(std_preds_on_lats)}")
	print(f" Lons: {np.nanmax(std_preds_on_lons)}\n")
	
	print(f"Upperbound: {np.nanmin(preds_upperbound)} \t {np.nanmax(preds_upperbound)}\n")
	print(f"Lowerbound: {np.nanmin(preds_lowerbound)} \t {np.nanmax(preds_lowerbound)}\n")
	
	print(np.where(avg_preds_on_time==np.nanmax(avg_preds_on_time)))
	__x_idx, __y_idx = np.where(avg_preds_on_time==np.nanmax(avg_preds_on_time))
	print(lats[__x_idx], lons[__y_idx])
	
	# ------------------------------------------------------------------------------------
	
	# compute the difference between real and predicted data
	avg_difference = (avg_real_descaled - avg_preds_descaled)
	avg_diff_on_lats = np.nanmean(avg_difference, axis=1)
	avg_diff_on_lons = np.nanmean(avg_difference, axis=0)
	
	std_difference = (std_real_descaled - std_preds_descaled)
	std_diff_on_lats = np.nanmean(std_difference, axis=1)
	std_diff_on_lons = np.nanmean(std_difference, axis=0)
	
	# define upperbound and lowerbound data for plotting the average on latitudes
	diff_upperbound = avg_diff_on_lats + std_diff_on_lats
	diff_lowerbound = avg_diff_on_lats - std_diff_on_lats
	
	print(f" Mean difference {avg_difference.shape}")
	print(f" Max: {np.nanmax(avg_difference)}")
	print(f" Min: {np.nanmin(avg_difference)}")
	print(f" Lats: {np.nanmax(avg_diff_on_lats)}")
	print(f" Lons: {np.nanmax(avg_diff_on_lons)}\n")
	
	print(f" Stdv difference {std_difference.shape}")
	print(f" Max: {np.nanmax(std_difference)}")
	print(f" Min: {np.nanmin(std_difference)}")
	print(f" Lats: {np.nanmax(std_diff_on_lats)}")
	print(f" Lons: {np.nanmax(std_diff_on_lons)}\n")
	
	print(f"Upperbound: {np.nanmin(diff_upperbound)} \t {np.nanmax(diff_upperbound)}\n")
	print(f"Lowerbound: {np.nanmin(diff_lowerbound)} \t {np.nanmax(diff_lowerbound)}\n")
	
	print(np.where(avg_difference==np.nanmax(avg_difference)))
	__x_idx_max, __y_idx_max= np.where(avg_difference==np.nanmax(avg_difference))
	print(lats[__x_idx_max], lons[__y_idx_max])
	
	# ------------------------------------------------------------------------------------
	
	exp_name = curr_exp['exp_name']
	
	plot_dataset_map(
		avg_target_data=avg_preds_descaled,
		avg_data_on_lats=avg_preds_on_lats,
		lowerbound_data=preds_lowerbound,
		upperbound_data=preds_upperbound,
		lats=lats,
		lons=lons,
		title = f'Predicted Burned Areas', # FCCI Burned Areas - {exp_name.upper()} Predictions',
		cmap = 'nipy_spectral_r') # 'CMRmap'
	
	
	plot_dataset_map(
		avg_target_data=avg_real_descaled,
		avg_data_on_lats=avg_real_on_lats,
		lowerbound_data=real_lowerbound,
		upperbound_data=real_upperbound,
		lats=lats,
		lons=lons,
		title = f'FCCI Burned Areas - {exp_name.upper()} Real',
		cmap = 'nipy_spectral_r') # 'CMRmap'
	
	
	plot_dataset_map(
		avg_target_data=avg_difference,
		avg_data_on_lats=avg_diff_on_lats,
		lowerbound_data=diff_lowerbound,
		upperbound_data=diff_upperbound,
		lats=lats,
		lons=lons,
		title = f'FCCI Burned Areas - {exp_name.upper()} Difference',
		cmap = 'gist_rainbow_r') # 'CMRmap'
